In [2128]:
"""
Скрипт для создания отчета по форме БРИТ
"""

'\nСкрипт для создания отчета по форме БРИТ\n'

In [2129]:
import pandas as pd

In [2130]:
def count_value(group:pd.Series,target_value:str):
    """
    Функция для группировки по конкретному значению
    Возвращает количество значений target_value в группе
    """
    # считаем сколько значений подходят по условие
    count_group = group.str.contains(target_value)
    return sum(count_group)


In [2131]:
dct_name_sheet = dict() # словарь где ключ это названия листа а значение датафрейм на основе которого был произведен подсчет

In [2132]:
df = pd.read_excel('data/Общий файл.xlsx',dtype=str)
df.fillna('Нет статуса',inplace=True) # заполняем Наны
group_main_df = pd.DataFrame(index=list(df['Файл'].unique()))
group_main_df.index.name = 'Файл'

C:\Users\1\anaconda3\lib\site-packages\openpyxl\worksheet\_read_only.py:79: UserWarning: Data Validation extension is not supported and will be removed
  for idx, row in parser.parse():


In [2133]:
# Отбрасываем на всякий случай отчисленных
df = df[df['Статус_Учёба']!= 'Отчислен']

In [2134]:
# Создаем датафрейм с сиротами
orphans_df = df[df['Статус_Сиротство'].isin(['гособеспечение + постинтернатное сопровождение','дети-сироты, находящиеся на полном государственном обеспечении',
                                                     'дети-сироты, находящиеся под опекой'])]

dct_name_sheet['Сироты'] = orphans_df # добавляем в словарь
orphans_group_df = orphans_df.groupby(by=['Файл']).agg({'Статус_Сиротство':'count'}) # создаем базовый 
group_main_df = group_main_df.join(orphans_group_df) # добавляем в свод
group_main_df.rename(columns={'Статус_Сиротство':'Дети-сироты'},inplace=True)

In [2135]:
group_main_df

,Дети-сироты
Файл,
Железнодорожного транспорта,9
Энергоснабжения,13
Металлообработки,29
Хоринский филиал,3


In [2136]:
# Создаем датафрейм с сиротами
invalid_df = df[df['Статус_Уровень_здоровья'].isin(['Инвалид детства','Инвалид 1,2,3, группы'])]

dct_name_sheet['Инвалиды'] = invalid_df # добавляем в словарь
invalid_group_df = invalid_df.groupby(by=['Файл']).agg({'Статус_Уровень_здоровья':'count'}) # создаем базовый 
group_main_df = group_main_df.join(invalid_group_df) # добавляем в свод
group_main_df.rename(columns={'Статус_Уровень_здоровья':'Инвалиды'},inplace=True)

In [2137]:
group_main_df

,Дети-сироты,Инвалиды
Файл,,
Железнодорожного транспорта,9,10
Энергоснабжения,13,10
Металлообработки,29,22
Хоринский филиал,3,3


In [2138]:
# Создаем датафрейм с получателями социальной стипендии
print(df.shape)
soc_benefit_df = df[df['Статус_Соц_стипендия'].isin(['да'])]
print(soc_benefit_df.shape)
dct_name_sheet['Соц. стипендия Все'] = soc_benefit_df # добавляем в словарь





(252, 55)
(30, 55)


In [2139]:
# получаем малоимущих
poor_soc_benefit_df = soc_benefit_df[soc_benefit_df['Статус_Соц_положение_семьи'].isin(['Малоимущая'])]
dct_name_sheet['Соц. стипендия малоим.'] = poor_soc_benefit_df
poor_soc_benefit_group_df = poor_soc_benefit_df.groupby(by=['Файл']).agg({'Статус_Соц_положение_семьи':'count'})
group_main_df = group_main_df.join(poor_soc_benefit_group_df) # добавляем в свод
group_main_df.rename(columns={'Статус_Соц_положение_семьи':'Соц. стипендия малоимущие'},inplace=True)


In [2140]:
# получаем детей сирот
orphans_soc_benefit_df = soc_benefit_df[soc_benefit_df['Статус_Сиротство'].isin(['гособеспечение + постинтернатное сопровождение','дети-сироты, находящиеся на полном государственном обеспечении',
                                                     'дети-сироты, находящиеся под опекой'])]
dct_name_sheet['Соц. стипендия сироты'] = orphans_soc_benefit_df
orphans_soc_benefit_group_df = orphans_soc_benefit_df.groupby(by=['Файл']).agg({'Статус_Сиротство':'count'})
group_main_df = group_main_df.join(orphans_soc_benefit_group_df) # добавляем в свод
group_main_df.rename(columns={'Статус_Сиротство':'Соц. стипендия дети-сироты'},inplace=True)

In [2141]:
# получаем инвалидов
invalid_soc_benefit_df = soc_benefit_df[soc_benefit_df['Статус_Уровень_здоровья'].isin(['Инвалид детства','Инвалид 1,2,3, группы'])]
dct_name_sheet['Соц. стипендия инвалиды'] = invalid_soc_benefit_df
invalid_soc_benefit_group_df = invalid_soc_benefit_df.groupby(by=['Файл']).agg({'Статус_Уровень_здоровья':'count'})
group_main_df = group_main_df.join(invalid_soc_benefit_group_df) # добавляем в свод
group_main_df.rename(columns={'Статус_Уровень_здоровья':'Соц. стипендия инвалиды'},inplace=True)

In [2142]:
# Создаем датафрейм с получателями бесплатного питания
print(df.shape)
eating_df = df[df['Статус_Питание'].isin(['получает компенсацию за питание','питается в БРИТ'])]
print(eating_df.shape)
dct_name_sheet['Питание все'] = eating_df # добавляем в словарь

(252, 55)
(60, 55)


In [2143]:
# получаем малоимущих
poor_eating_df = eating_df[eating_df['Статус_Соц_положение_семьи'].isin(['Малоимущая'])]
dct_name_sheet['Питание малоим.'] = poor_eating_df
poor_eating_group_df = poor_eating_df.groupby(by=['Файл']).agg({'Статус_Соц_положение_семьи':'count'})
group_main_df = group_main_df.join(poor_eating_group_df) # добавляем в свод
group_main_df.rename(columns={'Статус_Соц_положение_семьи':'Питание малоимущие'},inplace=True)

In [2144]:
# получаем сирот
orphans_eating_df = eating_df[eating_df['Статус_Сиротство'].isin(['гособеспечение + постинтернатное сопровождение','дети-сироты, находящиеся на полном государственном обеспечении',
                                                     'дети-сироты, находящиеся под опекой'])]
dct_name_sheet['Питание сироты'] = orphans_eating_df
orphans_eating_group_df = orphans_eating_df.groupby(by=['Файл']).agg({'Статус_Сиротство':'count'})
group_main_df = group_main_df.join(orphans_eating_group_df) # добавляем в свод
group_main_df.rename(columns={'Статус_Сиротство':'Питание сироты'},inplace=True)

In [2145]:
# получаем инвалидов
invalid_eating_df = eating_df[eating_df['Статус_Уровень_здоровья'].isin(['Инвалид детства','Инвалид 1,2,3, группы'])]
dct_name_sheet['Питание инвалиды'] = invalid_eating_df
invalid_eating_group_df = invalid_eating_df.groupby(by=['Файл']).agg({'Статус_Уровень_здоровья':'count'})
group_main_df = group_main_df.join(invalid_eating_group_df) # добавляем в свод
group_main_df.rename(columns={'Статус_Уровень_здоровья':'Питание инвалиды'},inplace=True)

In [2146]:
# получаем СВО
svo_eating_df = eating_df[eating_df['Статус_Родитель_СВО'].isin(['да'])]
dct_name_sheet['Питание СВО'] = svo_eating_df
svo_eating_group_df = svo_eating_df.groupby(by=['Файл']).agg({'Статус_Родитель_СВО':'count'})
group_main_df = group_main_df.join(svo_eating_group_df) # добавляем в свод
group_main_df.rename(columns={'Статус_Родитель_СВО':'Питание СВО'},inplace=True)

In [2147]:
# Создаем датафрейм с проживающими в общежитии
print(df.shape)
dormitory_df = df[df['Статус_Общежитие'].isin(['да'])]
print(dormitory_df.shape)
dct_name_sheet['Общежитие все'] = dormitory_df # добавляем в словарь
dormitory_group_df = dormitory_df.groupby(by=['Файл']).agg({'Статус_Общежитие':'count'})
group_main_df = group_main_df.join(dormitory_group_df) # добавляем в свод
group_main_df.rename(columns={'Статус_Общежитие':'Общежитие Всего'},inplace=True)

(252, 55)
(71, 55)


In [2148]:
# получаем не сирот и сохраняем для списка
not_orphans_dormitory_df = dormitory_df[~dormitory_df['Статус_Сиротство'].isin(['гособеспечение + постинтернатное сопровождение','дети-сироты, находящиеся на полном государственном обеспечении',
                                                     'дети-сироты, находящиеся под опекой'])]
dct_name_sheet['Общежитие кроме сирот'] = not_orphans_dormitory_df


In [2149]:
# получаем сирот
orphans_dormitory_df = dormitory_df[dormitory_df['Статус_Сиротство'].isin(['гособеспечение + постинтернатное сопровождение','дети-сироты, находящиеся на полном государственном обеспечении',
                                                     'дети-сироты, находящиеся под опекой'])]
dct_name_sheet['Питание сироты'] = orphans_dormitory_df
orphans_dormitory_group_df = orphans_dormitory_df.groupby(by=['Файл']).agg({'Статус_Сиротство':'count'})
group_main_df = group_main_df.join(orphans_dormitory_group_df) # добавляем в свод
group_main_df.rename(columns={'Статус_Сиротство':'Общежитие сироты'},inplace=True)

In [2150]:
# Считаем выпуск 2024
print(df.shape)
release_df = df[df['Статус_Выпуск'].isin(['да'])]
print(release_df.shape)
dct_name_sheet['Выпуск 2024'] = release_df # добавляем в словарь

(252, 55)
(80, 55)


In [2151]:
# Считаем сирот
release_orphans_df = release_df[release_df['Статус_Сиротство'].isin(['гособеспечение + постинтернатное сопровождение','дети-сироты, находящиеся на полном государственном обеспечении',
                                                     'дети-сироты, находящиеся под опекой'])]

dct_name_sheet['Выпуск сироты'] = release_orphans_df # добавляем в словарь
release_orphans_group_df = release_orphans_df.groupby(by=['Файл']).agg({'Статус_Сиротство':'count'}) # создаем базовый 
group_main_df = group_main_df.join(release_orphans_group_df) # добавляем в свод
group_main_df.rename(columns={'Статус_Сиротство':'Выпуск сироты'},inplace=True)

In [2152]:
# Считаем инвалидов
release_invalid_df = release_df[release_df['Статус_Уровень_здоровья'].isin(['Инвалид детства','Инвалид 1,2,3, группы'])]

dct_name_sheet['Выпуск инвалиды'] = release_invalid_df # добавляем в словарь
release_invalid_group_df = release_invalid_df.groupby(by=['Файл']).agg({'Статус_Уровень_здоровья':'count'}) # создаем базовый 
group_main_df = group_main_df.join(release_invalid_group_df) # добавляем в свод
group_main_df.rename(columns={'Статус_Уровень_здоровья':'Выпуск инвалиды'},inplace=True)

In [2153]:
group_main_df.fillna(0,inplace=True) # заполняем наны
group_main_df=group_main_df.astype(int) # приводим к инту
sum_row = group_main_df.sum(axis=0) # суммируем колонки
group_main_df.loc['Итого'] = sum_row # добавляем суммирующую колонку

In [2154]:
group_main_df

,Дети-сироты,Инвалиды,Соц. стипендия малоимущие,Соц. стипендия дети-сироты,Соц. стипендия инвалиды,Питание малоимущие,Питание сироты,Питание инвалиды,Питание СВО,Общежитие Всего,Общежитие сироты,Выпуск сироты,Выпуск инвалиды
Файл,,,,,,,,,,,,,
Железнодорожного транспорта,9,10,1,3,1,1,3,3,2,12,5,9,10
Энергоснабжения,13,10,0,4,0,0,4,3,3,17,7,0,0
Металлообработки,29,22,0,8,0,0,8,7,7,37,15,5,4
Хоринский филиал,3,3,0,1,1,2,1,1,2,5,2,3,3
Итого,54,45,1,16,2,3,16,14,14,71,29,17,17


In [2155]:
group_orphans_main_df = pd.DataFrame(index=list(df['Файл'].unique())) # Базовый датафрейм для сирот


In [2156]:
# Считаем общее количество
all_orphans_group_df = orphans_df.groupby(by=['Файл']).agg({'Статус_Сиротство':'count'}) # создаем базовый 
group_orphans_main_df = group_orphans_main_df.join(all_orphans_group_df) # добавляем в свод
group_orphans_main_df.rename(columns={'Статус_Сиротство':'Всего'},inplace=True)

In [2157]:
# считаем академ
akadem_orphans_df = orphans_df[orphans_df['Статус_Учёба'].str.contains('Академический отпуск')]
dct_name_sheet['Сироты академ'] = akadem_orphans_df
akadem_orphans_group_df = akadem_orphans_df.groupby(by=['Файл']).agg({'Статус_Сиротство':'count'})
group_orphans_main_df = group_orphans_main_df.join(akadem_orphans_group_df) # добавляем в свод
group_orphans_main_df.rename(columns={'Статус_Сиротство':'Из них в академическом отпуске'},inplace=True)

In [2158]:
# считаем постинтернатное сопровождение
postinternat_orphans_df = orphans_df[orphans_df['Статус_Сиротство'].str.contains('постинтернатное сопровождение')]
dct_name_sheet['Сироты постинтернат'] = postinternat_orphans_df
postinternat_orphans_group_df = postinternat_orphans_df.groupby(by=['Файл']).agg({'Статус_Сиротство':'count'})
group_orphans_main_df = group_orphans_main_df.join(postinternat_orphans_group_df) # добавляем в свод
group_orphans_main_df.rename(columns={'Статус_Сиротство':'Из них постинтернат'},inplace=True)

In [2159]:
# считаем сирот на гособеспечении
print(orphans_df.shape)
gos_orphans_df = orphans_df[orphans_df['Статус_Сиротство'].isin(['гособеспечение + постинтернатное сопровождение','дети-сироты, находящиеся на полном государственном обеспечении'
                                                     ])]
print(gos_orphans_df.shape)
dct_name_sheet['Сироты гособеспечение'] = gos_orphans_df
gos_orphans_group_df = gos_orphans_df.groupby(by=['Файл']).agg({'Статус_Сиротство':'count'})
group_orphans_main_df = group_orphans_main_df.join(gos_orphans_group_df) # добавляем в свод
group_orphans_main_df.rename(columns={'Статус_Сиротство':'Из них на гособеспечении'},inplace=True)

(54, 55)
(41, 55)


In [2160]:
# считаем сирот с опекой
print(orphans_df.shape)
custody_orphans_df = orphans_df[orphans_df['Статус_Сиротство'].isin(['дети-сироты, находящиеся под опекой'])]
print(custody_orphans_df.shape)
dct_name_sheet['Сироты опека'] = custody_orphans_df
custody_orphans_group_df = custody_orphans_df.groupby(by=['Файл']).agg({'Статус_Сиротство':'count'})
group_orphans_main_df = group_orphans_main_df.join(custody_orphans_group_df) # добавляем в свод
group_orphans_main_df.rename(columns={'Статус_Сиротство':'Из них под опекой'},inplace=True)

(54, 55)
(13, 55)


In [2161]:
# считаем сирот получающих питание
print(orphans_df.shape)
all_eating_orphans_df = orphans_df[orphans_df['Статус_Питание'].isin(['питается в БРИТ','получает компенсацию за питание',
                                                                     'получает компенсацию за питание + питается в БРИТ'])]
print(all_eating_orphans_df.shape)
dct_name_sheet['Сироты питание все'] = all_eating_orphans_df
all_eating_orphans_group_df = all_eating_orphans_df.groupby(by=['Файл']).agg({'Статус_Сиротство':'count'})
group_orphans_main_df = group_orphans_main_df.join(all_eating_orphans_group_df) # добавляем в свод
group_orphans_main_df.rename(columns={'Статус_Сиротство':'Питается всего'},inplace=True)

(54, 55)
(17, 55)


In [2162]:
# считаем сирот получающих питание в брит
print(orphans_df.shape)
brit_eating_orphans_df = orphans_df[orphans_df['Статус_Питание'].isin(['питается в БРИТ'])]
print(brit_eating_orphans_df.shape)
dct_name_sheet['Сироты питание БРИТ'] = brit_eating_orphans_df
brit_eating_orphans_group_df = brit_eating_orphans_df.groupby(by=['Файл']).agg({'Статус_Сиротство':'count'})
group_orphans_main_df = group_orphans_main_df.join(brit_eating_orphans_group_df) # добавляем в свод
group_orphans_main_df.rename(columns={'Статус_Сиротство':'Питается в БРИТ'},inplace=True)

(54, 55)
(1, 55)


In [2163]:
# считаем сирот получающих компенсацию
print(orphans_df.shape)
compens_eating_orphans_df = orphans_df[orphans_df['Статус_Питание'].isin(['получает компенсацию за питание'])]
print(compens_eating_orphans_df.shape)
dct_name_sheet['Сироты питание компенсация'] = compens_eating_orphans_df
compens_eating_orphans_group_df = compens_eating_orphans_df.groupby(by=['Файл']).agg({'Статус_Сиротство':'count'})
group_orphans_main_df = group_orphans_main_df.join(compens_eating_orphans_group_df) # добавляем в свод
group_orphans_main_df.rename(columns={'Статус_Сиротство':'Получает компенсацию'},inplace=True)

(54, 55)
(15, 55)


In [2164]:
# считаем сирот получающих компенсацию + питание в брит
print(orphans_df.shape)
brit_compens_eating_orphans_df = orphans_df[orphans_df['Статус_Питание'].isin(['получает компенсацию за питание + питается в БРИТ'])]
print(brit_compens_eating_orphans_df.shape)
dct_name_sheet['Сироты питание БРИТ+компенсация'] = brit_compens_eating_orphans_df
brit_compens_eating_orphans_group_df = brit_compens_eating_orphans_df.groupby(by=['Файл']).agg({'Статус_Сиротство':'count'})
group_orphans_main_df = group_orphans_main_df.join(brit_compens_eating_orphans_group_df) # добавляем в свод
group_orphans_main_df.rename(columns={'Статус_Сиротство':'БРИТ+компенсация'},inplace=True)

(54, 55)
(1, 55)


In [2165]:
group_orphans_main_df.fillna(0,inplace=True) # заполняем наны
group_orphans_main_df=group_orphans_main_df.astype(int) # приводим к инту
sum_row = group_orphans_main_df.sum(axis=0) # суммируем колонки
group_orphans_main_df.loc['Итого'] = sum_row # добавляем суммирующую колонку

In [2166]:
group_orphans_main_df

,Всего,Из них в академическом отпуске,Из них постинтернат,Из них на гособеспечении,Из них под опекой,Питается всего,Питается в БРИТ,Получает компенсацию,БРИТ+компенсация
Железнодорожного транспорта,9,2,4,7,2,3,0,3,0
Энергоснабжения,13,3,6,10,3,4,0,4,0
Металлообработки,29,7,14,22,7,8,0,8,0
Хоринский филиал,3,2,2,2,1,2,1,0,1
Итого,54,14,26,41,13,17,1,15,1


In [2167]:
# отчет по учетам

In [2168]:
group_accounting_main_df = pd.DataFrame(index=list(df['Файл'].unique())) # Базовый датафрейм для учетов

In [2169]:
# Считаем КДН
print(df.shape)
kdn_accounting_df = df[df['Статус_КДН'].isin(['состоит'])]
print(kdn_accounting_df.shape)
dct_name_sheet['КДН'] = kdn_accounting_df
kdn_group_accounting_df = kdn_accounting_df.groupby(by=['Файл']).agg({'Статус_КДН':'count'})
group_accounting_main_df = group_accounting_main_df.join(kdn_group_accounting_df) # добавляем в свод


(252, 55)
(13, 55)


In [2170]:
# Считаем ПДН
print(df.shape)
pdn_accounting_df = df[df['Статус_ПДН'].isin(['состоит'])]
print(pdn_accounting_df.shape)
dct_name_sheet['ПДН'] = pdn_accounting_df
pdn_group_accounting_df = pdn_accounting_df.groupby(by=['Файл']).agg({'Статус_ПДН':'count'})
group_accounting_main_df = group_accounting_main_df.join(pdn_group_accounting_df) # добавляем в свод

(252, 55)
(42, 55)


In [2171]:
# Слаживаем колонки и удаляем лишнее
group_accounting_main_df.fillna(0,inplace=True)
group_accounting_main_df['Учет КДН, ПДН'] = group_accounting_main_df['Статус_КДН'] + group_accounting_main_df['Статус_ПДН']
group_accounting_main_df.drop(columns=['Статус_КДН','Статус_ПДН'],inplace=True)

In [2172]:
# Считаем внутренний учет
print(df.shape)
inside_accounting_df = df[df['Статус_Внутр_учет'].isin(['состоит'])]
print(inside_accounting_df.shape)
dct_name_sheet['Внутр_учет'] = inside_accounting_df
inside_group_accounting_df = inside_accounting_df.groupby(by=['Файл']).agg({'Статус_Внутр_учет':'count'})
group_accounting_main_df = group_accounting_main_df.join(inside_group_accounting_df) # добавляем в свод
group_accounting_main_df.rename(columns={'Статус_Внутр_учет':'Внутренний учет'},inplace=True)

(252, 55)
(52, 55)


In [2173]:
# Считаем самовольный уход
print(df.shape)
awol_accounting_df = df[df['Статус_Самовольный_уход'].isin(['да'])]
print(awol_accounting_df.shape)
dct_name_sheet['Самовольный уход'] = awol_accounting_df
awol_group_accounting_df = awol_accounting_df.groupby(by=['Файл']).agg({'Статус_Самовольный_уход':'count'})
group_accounting_main_df = group_accounting_main_df.join(awol_group_accounting_df) # добавляем в свод
group_accounting_main_df.rename(columns={'Статус_Самовольный_уход':'Самовольный уход'},inplace=True)

(252, 55)
(16, 55)


In [2174]:
# Считаем самовольный уход
print(df.shape)
sop_accounting_df = df[df['Статус_Соц_положение_семьи'].isin(['СОП'])]
print(sop_accounting_df.shape)
dct_name_sheet['СОП'] = sop_accounting_df
sop_group_accounting_df = sop_accounting_df.groupby(by=['Файл']).agg({'Статус_Соц_положение_семьи':'count'})
group_accounting_main_df = group_accounting_main_df.join(sop_group_accounting_df) # добавляем в свод
group_accounting_main_df.rename(columns={'Статус_Соц_положение_семьи':'СОП'},inplace=True)

(252, 55)
(29, 55)


In [2175]:
group_accounting_main_df.fillna(0,inplace=True) # заполняем наны
group_accounting_main_df=group_accounting_main_df.astype(int) # приводим к инту
sum_row = group_accounting_main_df.sum(axis=0) # суммируем колонки
group_accounting_main_df.loc['Итого'] = sum_row # добавляем суммирующую колонку

In [2176]:
group_accounting_main_df

,"Учет КДН, ПДН",Внутренний учет,Самовольный уход,СОП
Железнодорожного транспорта,9,8,3,5
Энергоснабжения,13,12,4,6
Металлообработки,29,28,8,15
Хоринский филиал,4,4,1,3
Итого,55,52,16,29
